In [ ]:
import re
import nltk
nltk.download('punkt')#divides a text into a list of sentences
nltk.download('wordnet')
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import warnings
warnings.filterwarnings("ignore")

def read_data(file):
    data = []
    with open('/content/text 2.txt',) as f:
        for line in f:
            line = line.strip()
            label = ' '.join(line[1:line.find("]")].strip().split())
            text = line[line.find("]") + 1:].strip()
            data.append([label, text])
    return data

file = 'text 2.txt'
data = read_data(file)
print("Number of instances: {}".format(len(data)))
def ngram(token, n):
   output = []
   for i in range(n-1, len(token)):
       ngram = ' '.join(token[i-n+1:i+1])
       output.append(ngram)
   return output
def create_feature(text, nrange=(1, 1)):
   text_features = []
   text = text.lower()
   text_alphanum = re.sub('[^a-z0-9#]', ' ', text)
   for n in range(nrange[0], nrange[1]+1):
     text_features += ngram(text_alphanum.split(), n)
   text_punc = re.sub('[a-z0-9]', ' ', text)
   text_features += ngram(text_punc.split(), 1)
   return Counter(text_features)
def convert_label(item, name):
  items = list(map(float, item.split()))
  label = ""
  for idx in range(len(items)):
    if items[idx] == 1:
       label += name[idx] + " "
  return label.strip()
emotions = ["joy", 'fear', "anger", "sadness", "disgust",
"shame", "guilt" ,"surprise" , "anticipation"]
X_all = []
y_all = []
for label, text in data:
   y_all.append(convert_label(label, emotions))
   X_all.append(create_feature(text, nrange=(1, 4)))
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all,
test_size = 0.2, random_state = 123)
def train_test(clf, X_train, X_test, y_train, y_test):
  clf.fit(X_train, y_train)
  train_acc = accuracy_score(y_train, clf.predict(X_train))
  test_acc = accuracy_score(y_test, clf.predict(X_test))
  return train_acc, test_acc
from sklearn.feature_extraction import DictVectorizer

vectorizer = DictVectorizer(sparse = True)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
svc = SVC()
lsvc = LinearSVC(random_state=123)
rforest = RandomForestClassifier(random_state=123)
dtree = DecisionTreeClassifier()
clifs = [svc, lsvc, rforest, dtree]

# train and test them
print("| {:25} | {} | {} |".format("Classifier", "Training Accuracy", "Test Accuracy"))
print("| {} | {} | {} |".format("-"*25, "-"*17, "-"*13))
for clf in clifs:
    clf_name = clf.__class__.__name__
    train_acc, test_acc = train_test(clf, X_train, X_test, y_train, y_test)
    print("| {:25} | {:17.7f} | {:13.7f} |".format(clf_name, train_acc, test_acc))

emoji_dict = {"joy": "😂", "fear": "😱", "anger": "😡",
              "sadness": "😥", "disgust": "🤢", "shame": "😳",
              "guilt": "😓", "surprise": "😮", "anticipation": "😃"}

t1 ="During the period of falling in love, each time that we met and especially when we had not met for a long time."
t2 ="When I was involved in a traffic accident."
t3 ="When I was driving home after several days of hard work, there was a motorist ahead of me who was driving at 50 km/hour and refused, despite his low speed, to let me overtake."
t4 ="When I lost the person who meant the most to me."
t5 ="The time I knocked a deer down - the sight of the animal's injuries and helplessness. The realization that the animal was so badly hurt that it had to be put down, and when the animal screamed at the moment of death."
t6 = "When I did not speak the truth."
t7 ="When I caused problems for somebody because he could not keep the appointed time and this led to various consequences."
t8 ="Opening the gift, I found a plane ticket to my dream destination."
t9 ="The countdown has begun, and I can't wait for the much-anticipated reunion with my best friends."
texts = [t1, t2, t3, t4 ,t5 ,t6 ,t7 ,t8 ,t9]


for text in texts:
  features = create_feature(text, nrange=(1, 4))
  features = vectorizer.transform(features)
  prediction = clf.predict(features)[0]
  print( text,emoji_dict[prediction])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Number of instances: 50
| Classifier                | Training Accuracy | Test Accuracy |
| ------------------------- | ----------------- | ------------- |
| SVC                       |         0.7750000 |     0.0000000 |
| LinearSVC                 |         0.9750000 |     0.3000000 |
| RandomForestClassifier    |         0.9750000 |     0.1000000 |
| DecisionTreeClassifier    |         0.9750000 |     0.2000000 |
During the period of falling in love, each time that we met and especially when we had not met for a long time. 😂
When I was involved in a traffic accident. 😱
When I was driving home after several days of hard work, there was a motorist ahead of me who was driving at 50 km/hour and refused, despite his low speed, to let me overtake. 😡
When I lost the person who meant the most to me. 😥
The time I knocked a deer down - the sight of the animal's injuries and helplessness. The realization that the animal was so badly hurt that it had to be put down, and when the animal screamed